<a href="https://colab.research.google.com/github/anjanadevi05/Defect_Detection/blob/main/defect_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from os import path
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#importing required libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator , load_img ,img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
import zipfile
import os

# Assume your file is named 'your_file.zip'
filename = '/content/casting_data.zip'

# Unzip the file
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('extracted_folder')  # Specify the folder where you want to extract the files

# List the extracted files
os.listdir('extracted_folder')


['casting_data']

In [ ]:
import os

# Define the base directory after extracting your zip file
my_data_dir = '/content/extracted_folder/casting_data/'  # Adjust according to your actual path

# Define train and test paths
train_path = os.path.join(my_data_dir, 'train')
test_path = os.path.join(my_data_dir, 'test')

# Parameters for image processing
image_shape = (300, 300, 1)
batch_size = 32

# Verify the structure by listing contents of the directories
print("Train directory contents:", os.listdir(train_path))
print("Test directory contents:", os.listdir(test_path))


Train directory contents: ['ok_front', 'def_front']
Test directory contents: ['ok_front', 'def_front']


In [ ]:
image_gen = ImageDataGenerator(rotation_range=75,
                               width_shift_range=0,
                               height_shift_range=0,
                               rescale=1/255,
                               shear_range=0,
                               zoom_range=0,
                               horizontal_flip=True,
                               fill_mode='nearest'
                              )

In [ ]:
model = Sequential()

In [ ]:
train_set = image_gen.flow_from_directory(train_path,
                                               target_size=image_shape[:2],
                                                color_mode="grayscale",
                                               batch_size=batch_size,
                                               class_mode='binary',shuffle=True)

test_set = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               color_mode="grayscale",
                                               batch_size=batch_size,
                                               class_mode='binary',shuffle=False)

Found 6633 images belonging to 2 classes.
Found 715 images belonging to 2 classes.


In [ ]:
image_gen = ImageDataGenerator(rescale=1/255)

In [ ]:
train_set.class_indices

{'def_front': 0, 'ok_front': 1}

CNN


In [ ]:
model = Sequential()

model.add(Conv2D(filters=8, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=16, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=16, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(224))
model.add(Activation('relu'))
# Last layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss',patience=3)

In [ ]:
results = model.fit(train_set,epochs=20, # Use fit instead of fit_generator
                              validation_data=test_set,
                             callbacks=[early_stop])

Epoch 1/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - accuracy: 0.6539 - loss: 0.6640 - val_accuracy: 0.8336 - val_loss: 0.3340
Epoch 2/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - accuracy: 0.9224 - loss: 0.2141 - val_accuracy: 0.8727 - val_loss: 0.2650
Epoch 3/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 318s 2s/step - accuracy: 0.9671 - loss: 0.0923 - val_accuracy: 0.9846 - val_loss: 0.0597
Epoch 4/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 315s 2s/step - accuracy: 0.9752 - loss: 0.0754 - val_accuracy: 0.9944 - val_loss: 0.0273
Epoch 5/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - accuracy: 0.9834 - loss: 0.0522 - val_accuracy: 0.9902 - val_loss: 0.0216
Epoch 6/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.9919 - loss: 0.0319 - val_accuracy: 0.9902 - val_loss: 0.0211
Epoch 7/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 324s 2s/step - accuracy: 0.9964 - loss: 0.0156 - val_accuracy: 0.9944 - val_loss: 0.0151
Epoch 8/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 314s 2s/step - accuracy: 0.9947 - loss: 0.0189 - val_accu

In [ ]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_set)

print(f"Test Accuracy: {accuracy * 100:.2f}%")


23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 516ms/step - accuracy: 0.9966 - loss: 0.0182
Test Accuracy: 99.44%


ResNet


In [ ]:
!pip install transformers torch


In [ ]:
from transformers import ResNetForImageClassification, AutoImageProcessor
from tensorflow.keras.optimizers import Adam
from transformers import AdamW
from torchvision import transforms
import torch
import numpy as np
from PIL import Image
import tensorflow as tf

In [ ]:
# Load the ResNet model from Hugging Face and the processor for preprocessing
processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50", num_labels=2, ignore_mismatched_sizes = True)
# Binary classification # Added ignore_mismatched_sizes = True to handle the mismatch in the number of classes in the classifier layer

preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([2, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import os

class ResNetDataGenerator(Dataset):
    def __init__(self, image_directory, processor):
        self.image_directory = image_directory
        self.processor = processor
        self.image_files = []
        self.labels = []

        for label, sub_dir in enumerate(['def_front', 'ok_front']):
            sub_dir_path = os.path.join(self.image_directory, sub_dir)
            if os.path.exists(sub_dir_path):
                for file in os.listdir(sub_dir_path):
                    file_path = os.path.join(sub_dir_path, file)
                    if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                        self.image_files.append(file_path)
                        self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        label = self.labels[idx]

        # Use the processor to preprocess the image
        image = Image.open(image_path).convert("RGB")  # Ensure it's in RGB format
        inputs = self.processor(images=image, return_tensors="pt")  # Use processor for preprocessing

        return inputs['pixel_values'].squeeze(0), torch.tensor(label)  # Return image and label


In [ ]:
from torch.utils.data import DataLoader

train_directory = '/content/extracted_folder/casting_data/train'
test_directory = '/content/extracted_folder/casting_data/test'

# Create instances of the data generator
train_data_gen = ResNetDataGenerator(train_directory, processor)
test_data_gen = ResNetDataGenerator(test_directory, processor)

train_loader = DataLoader(train_data_gen, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data_gen, batch_size=32, shuffle=False)


In [ ]:
import torch.optim as optim
from tqdm import tqdm
import torch.nn as nn

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

# Early stopping class
class EarlyStopping:
    def __init__(self, patience=3, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.best_loss = None
        self.counter = 0

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                return True  # Stop training
        else:
            self.best_loss = val_loss
            self.counter = 0
        return False  # Continue training

# Create an instance of EarlyStopping
early_stopping = EarlyStopping(patience=3, verbose=True)

# Training loop
num_epochs = 5  # Set the number of epochs

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)  # Move to GPU if available
        optimizer.zero_grad()

        outputs = model(images).logits  # Forward pass
        loss = nn.CrossEntropyLoss()(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

    # Calculate validation loss for early stopping
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            loss = nn.CrossEntropyLoss()(outputs, labels)
            val_loss += loss.item()

    val_loss /= len(test_loader)  # Average validation loss
    print(f'Validation Loss: {val_loss:.4f}')

    # Check for early stopping
    if early_stopping(val_loss):
        print("Early stopping triggered")
        break  # Stop training


100%|██████████| 208/208 [01:36<00:00,  2.15it/s]


Epoch [1/5], Loss: 0.0048
Validation Loss: 0.0113


100%|██████████| 208/208 [01:34<00:00,  2.20it/s]


Epoch [2/5], Loss: 0.0049
Validation Loss: 0.0055


100%|██████████| 208/208 [01:35<00:00,  2.19it/s]


Epoch [3/5], Loss: 0.0037
Validation Loss: 0.0085
EarlyStopping counter: 1 out of 3


100%|██████████| 208/208 [01:34<00:00,  2.19it/s]


Epoch [4/5], Loss: 0.0019
Validation Loss: 0.0083
EarlyStopping counter: 2 out of 3


100%|██████████| 208/208 [01:35<00:00,  2.18it/s]


Epoch [5/5], Loss: 0.0019
Validation Loss: 0.0046


In [ ]:
# Set the model to evaluation mode
model.eval()

# Initialize variables to track the correct predictions and total samples
correct_predictions = 0
total_samples = 0

# Disable gradient calculation for efficiency
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move to GPU if available

        outputs = model(images).logits  # Get model outputs
        _, predicted = torch.max(outputs, 1)  # Get predicted labels
        total_samples += labels.size(0)  # Update total samples
        correct_predictions += (predicted == labels).sum().item()  # Count correct predictions

# Calculate test accuracy
test_accuracy = correct_predictions / total_samples
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Test Accuracy: 99.72%


In [ ]:
print(train_data_gen.class_to_idx)
print(test_data_gen.class_to_idx)